In [1]:
# NBVAL_IGNORE_OUTPUT
import lineapy

x = 100
y = 10

z = [x]

dag_path = lineapy.save(z, "z").to_airflow()

Added Airflow DAG named 'z'. Start a run from the Airflow UI or CLI.


In [2]:
print(dag_path.read_text())

from os import chdir

from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.python_operator import PythonOperator


chdir("tests/notebook")


def z():
    x = 100
    z = [x]


default_dag_args = {
    "owner": "airflow",
    "retries": 2,
    "start_date": days_ago(1),
}

dag = DAG(
    dag_id="z_dag",
    schedule_interval="*/15 * * * *",  # Every 15 minutes
    max_active_runs=1,
    catchup=False,
    default_args=default_dag_args,
)
z = PythonOperator(
    dag=dag,
    task_id="z_task",
    python_callable=z,
)

